In [1]:
# -*- coding:utf-8 -*-
# 第一行必须有，否则报中文字符非ascii码错误
import urllib.request
from urllib.parse import quote
import json
import pandas as pd

#improved version: put the adcode and citycode into database, 
#input citycode first, then 
#select adcode
#from dataset
#where citycode = ...
#adcode = [440103, 440104, 440105, 440106, 440111, 440112, 440113, 440114, 440115, 440117, 440118]

In [2]:
#import xlsx file
def xlsx_to_df(xlsxname, sheetname):
    xlsx = pd.ExcelFile(xlsxname)
    df = pd.read_excel(xlsx, sheet_name=sheetname, dtype = str)
    df.fillna('', inplace = True)
    return df

In [3]:
def getcitycode(cityname, dataframe):
    try:
        cc = dataframe[dataframe['中文名'] == cityname]['citycode']
        cityc = cc.iloc[0]
        print('the citycode of ' + cityname + ': ' + cityc)
        adcode = getadcode(dataframe, cityc, cityname)
        return adcode
    except:
        print('Not exist!')

In [4]:
def getadcode(dataframe, citycode, cityname):
    ac = dataframe[(dataframe['citycode'] == citycode) & (dataframe['中文名'] != cityname)]
    #create a list that contains all district name
    test_list = list(ac['中文名'])
    test_list_new = []
    for i in test_list:
        if '市辖区' in i:
            #to get a list including 市辖区
            test_list_new.append(i)
    
    ac = ac[~ac['中文名'].isin(test_list_new)]
    print(ac)
    #to get all valid adcode
    adcodelist = list(ac['adcode'])
    if adcodelist == None:
        print('No adcode!')
    return adcodelist

In [5]:
url = 'http://restapi.amap.com/v3/config/district?'


def getlnglat(address, key):

    uri = url + 'keywords=' + address + '&key=' + key + '&subdistrict=1' + '&extensions=all'
    #print(uri)

    # 访问链接后，api会回传给一个json格式的数据
    temp = urllib.request.urlopen(uri)
    temp = json.loads(temp.read())

    # polyline是坐标，name是区域的名字
    #有两个district 选第一个
    #Data = all polyline
    Data = temp["districts"][0]['polyline']
    name = temp['districts'][0]['name']

    lngs = []
    lats = []
    points = []
    
    #iterate each point 
    for line in str(Data).split(";"):
        
        #if using '|' for spliting
        if len(line.split("|")) > 1:
            #split again
            for uu in line.split("|"):
                if float(uu.split(",")[0]) != None:
                    lngs.append(float(uu.split(",")[0])) #longitude
                    lats.append(float(uu.split(",")[1])) #latitude
                    points.append([float(uu.split(",")[0]), float(uu.split(",")[1])])
        #split by ';' as usual
        else:
            if float(line.split(",")[0]) != None:
                lngs.append(float(line.split(",")[0]))
                lats.append(float(line.split(",")[1]))
                points.append([float(line.split(",")[0]), float(line.split(",")[1])])
    #put the site into DateFrame, then import as csv file
    
    #print(points)
    #print(max(lngs), min(lngs), max(lats), min(lats))
    
    #如果需要把polyline存到csv的话
    #frame = pd.DataFrame({'Longtitude': lngs, 'Latitude': lats, 'Points': points})
    #frame.to_csv('{0}.csv'.format(name))
    
    return max(lngs), min(lngs), max(lats), min(lats)


In [6]:
import numpy as np
import pandas as pd
import xlwt

In [7]:
#以最大最小经纬度围成的矩形，通过center point 分成四个矩形
#把四个矩形的起始&终止经纬度 存入csv中

def generate_grids(start_long,start_lat,end_long,end_lat, adcode):
    
    assert start_long < end_long,'需要从左上到右下设置经度，start的经度应小于end的经度'
    assert start_lat > end_lat,'需要从左上到右下设置纬度，start的纬度应大于end的纬度'
    #assert resolution>0,'resolution应大于0'
    
    centerlng = round(float((end_long+start_long)/2),6)
    centerlat = round(float((end_lat+start_lat)/2),6)
    minlng = round(float(start_long),6)
    maxlng = round(float(end_long),6)
    minlat = round(float(end_lat),6)
    maxlat = round(float(start_lat),6)
    
    book = xlwt.Workbook(encoding='utf-8', style_compression=0)
    sheet = book.add_sheet('GRID', cell_overwrite_ok=True)
    
    sheet.write(0, 0, '起始经度')
    sheet.write(0, 1, '起始纬度')
    sheet.write(0, 2, '终止经度')
    sheet.write(0, 3, '终止纬度')
    
    #first grid -- upper-left
    sheet.write(1, 0, minlng)
    sheet.write(1, 1, maxlat)
    sheet.write(1, 2, centerlng)
    sheet.write(1, 3, centerlat)
    
    #second grid -- upper-right
    sheet.write(2, 0, centerlng)
    sheet.write(2, 1, maxlat)
    sheet.write(2, 2, maxlng)
    sheet.write(2, 3, centerlat)
    
    #third grid -- lower-left
    sheet.write(3, 0, minlng)
    sheet.write(3, 1, centerlat)
    sheet.write(3, 2, centerlng)
    sheet.write(3, 3, minlat)
    
    #fourth grid -- lower-right
    sheet.write(4, 0, centerlng)
    sheet.write(4, 1, centerlat)
    sheet.write(4, 2, maxlng)
    sheet.write(4, 3, minlat)
    
    # 最后，将以上操作保存到指定的Excel文件中
    book.save(r'' + "GRID_"+ str(adcode) + '.xls')

In [8]:
#key = 'c46811a81734366e089404d4b270049e'
excel = input('EXCEL File: ')
excel = str(excel)+'.xlsx'
sheetname = input('sheetname: ')
sheetname = str(sheetname)
cityname = input('Please input the name of cityname: ')
cityname = str(cityname)
key = input('Please input your key: ')
key = str(key)

dataframe = xlsx_to_df(excel, sheetname)
adcode = getcitycode(cityname, dataframe)
for ad in adcode:
    # a, b, c, d = max(lngs), min(lngs), max(lats), min(lats)
    a, b, c, d = getlnglat(ad, key)
    generate_grids(b, c, a, d, ad)
    
print('Finish')

EXCEL File: adcode
sheetname: Sheet1
Please input the name of cityname: 广州市
Please input your key: c46811a81734366e089404d4b270049e
the citycode of 广州市: 020
      中文名  adcode citycode
2112  荔湾区  440103      020
2113  越秀区  440104      020
2114  海珠区  440105      020
2115  天河区  440106      020
2116  白云区  440111      020
2117  黄埔区  440112      020
2118  番禺区  440113      020
2119  花都区  440114      020
2120  南沙区  440115      020
2121  从化区  440117      020
2122  增城区  440118      020
Finish
